<h2>Fazendo web scraping com IMDb</h2>

In [ ]:
# ! pip install bs4

In [1]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import re

In [52]:
# url com a listagem de filmes de Al Pacino
# url = 'https://www.imdb.com/name/nm0000199/?nmdp=1&ref_=nm_ql_4'
url = 'https://www.imdb.com/name/nm0000199/?ref_=nm_ql_4'

# fazendo a conexão
con = requests.get(url)
con.status_code

# criando objeto beautifulsoup
# o conteúdo html da página
soup = BeautifulSoup(con.content,"html.parser")

In [26]:
# criando objeto beautifulsoup
# o conteúdo html da página
soup = BeautifulSoup(con.content,"html.parser")

In [51]:
# print(soup.prettify())

In [41]:
# pegando tudo relacionado a filmes
nomeFilmes = [tag.find('a') for tag in soup.findAll('div', class_='filmo-row')]

In [42]:
# tratando caracteres indesejados
anoFilmes = [tag.find('span') for tag in soup.findAll('div', class_='filmo-row')]

anos = []
for num in anoFilmes:
    id = anoFilmes.index(num)
    c = re.findall('\d+', str(anoFilmes[id]))
    c = re.sub("\['", '', str(c))
    c = re.sub("\']",'',str(c))
    c = re.sub("\'\,",' -',str(c))
    c = re.sub("\ '",' ',str(c))
    anos.append(c)

In [50]:
# anos

In [44]:
# usando re para pegar tudo o que está entre "" para gerar o link do filme
linkPart = 'http://imdb.com'
linksFilmes = []
for num in nomeFilmes:
    id = nomeFilmes.index(num)
    a = re.findall(r'"(.*?)"', str(nomeFilmes[id]))
    a = re.sub("\['", '', str(a))
    a = re.sub("\']", '', str(a))
    a = linkPart + str(a)
    linksFilmes.append(a)

In [45]:
titulosFilmes = []
for num in nomeFilmes:
    id = nomeFilmes.index(num)
    b = re.findall(r'>(.*?)<', str(nomeFilmes[id]))
    b = re.sub("\['", '', str(b))
    b = re.sub("\']", '', str(b))
    b = re.sub('\["', '', str(b))
    b = re.sub('\"]', '', str(b))
    titulosFilmes.append(b)

In [46]:
def pegaAno(soup):
    anoFilme = [tag.find('a') for tag in soup2.findAll('h1', itemprop='name')]
    anoFilme = re.findall(r'>(.*?)<', str(anoFilme))
    if len(anoFilme):
        return int(anoFilme[0])
    else:
        return None

In [53]:
def pegaNotaIMDB(link):
    for i in link:
        con2 = requests.get(link)
        soup2= BeautifulSoup(con2.content, "html.parser")
        ratFilme = [tag.find('span', itemprop='ratingValue') for tag in soup2.findAll('div', class_='ratingValue')]
        ratFilme = re.findall(r'>(.*?)<', str(ratFilme))
        if len(ratFilme):
            return str(ratFilme[0])
        else:
            return None

In [49]:
# print(linksFilmes)

In [91]:
# funções para pegar informações do filme
qtdFilmAct = 80
film = {}
listaFilmes = []

for i in range(qtdFilmAct):
    con2 = requests.get(linksFilmes[i])
    soup2 = BeautifulSoup(con2.content, "html.parser")
    con2.close()

    film['Titulo']            = titulosFilmes[i]
    film['Link']              = linksFilmes[i]
    film['Ano']               = anos[i]
    #film['Genero']            = pegaGenero(linksFilmes[i])
    film['Nota IMDB']         = pegaNotaIMDB(linksFilmes[i])
    listaFilmes.append(film.copy())

In [92]:
listaFilmes

[{'Titulo': 'King Lear',
  'Link': 'http://imdb.com/title/tt1371129/?ref_=nm_flmg_act_1',
  'Ano': '[]',
  'Nota IMDB': None},
 {'Titulo': 'Axis Sally',
  'Link': 'http://imdb.com/title/tt7050946/?ref_=nm_flmg_act_2',
  'Ano': '2020',
  'Nota IMDB': None},
 {'Titulo': 'The Irishman',
  'Link': 'http://imdb.com/title/tt1302006/?ref_=nm_flmg_act_3',
  'Ano': '2019',
  'Nota IMDB': None},
 {'Titulo': 'Once Upon a Time in Hollywood',
  'Link': 'http://imdb.com/title/tt7131622/?ref_=nm_flmg_act_4',
  'Ano': '2019',
  'Nota IMDB': None},
 {'Titulo': 'Paterno',
  'Link': 'http://imdb.com/title/tt2388986/?ref_=nm_flmg_act_5',
  'Ano': '2018',
  'Nota IMDB': '6.5'},
 {'Titulo': 'Letras da Morte',
  'Link': 'http://imdb.com/title/tt3011960/?ref_=nm_flmg_act_6',
  'Ano': '2017',
  'Nota IMDB': '5.2'},
 {'Titulo': 'The Pirates of Somalia',
  'Link': 'http://imdb.com/title/tt5126922/?ref_=nm_flmg_act_7',
  'Ano': '2017',
  'Nota IMDB': '6.8'},
 {'Titulo': 'Má Conduta',
  'Link': 'http://imdb.com/ti

In [93]:
dffilme = pd.DataFrame(listaFilmes, columns=[
    'Titulo', 'Link', 'Ano', 'Nota IMDB'
])
dffilme

,Titulo,Link,Ano,Nota IMDB
0,King Lear,http://imdb.com/title/tt1371129/?ref_=nm_flmg_...,[],None
1,Axis Sally,http://imdb.com/title/tt7050946/?ref_=nm_flmg_...,2020,None
2,The Irishman,http://imdb.com/title/tt1302006/?ref_=nm_flmg_...,2019,None
3,Once Upon a Time in Hollywood,http://imdb.com/title/tt7131622/?ref_=nm_flmg_...,2019,None
4,Paterno,http://imdb.com/title/tt2388986/?ref_=nm_flmg_...,2018,6.5
5,Letras da Morte,http://imdb.com/title/tt3011960/?ref_=nm_flmg_...,2017,5.2
6,The Pirates of Somalia,http://imdb.com/title/tt5126922/?ref_=nm_flmg_...,2017,6.8
7,Má Conduta,http://imdb.com/title/tt3658772/?ref_=nm_flmg_...,2016,5.3
8,Não Olhe para Trás,http://imdb.com/title/tt1772288/?ref_=nm_flmg_...,2015,7.0
9,Manglehorn,http://imdb.com/title/tt2893490/?ref_=nm_flmg_...,2014,5.6
